In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22727066
paper_name = 'jaime_nislow_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/12864_2012_4394_MOESM2_ESM.xlsx', sheet_name='Table_S1', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5389 x 5


In [10]:
original_data.head()

,ORF,Fitness defect score,SGD_orf_name,Gene,SGD_description
0,YAL001C::chr1_1,-1.814,YAL001C,TFC3,Largest of six subunits of the RNA polymerase ...
1,YAL003W::chr1_1,-0.640,YAL003W,EFB1,Translation elongation factor 1 beta; stimulat...
2,YAL004W::chr1_1,-0.139,YAL004W,NaN,Dubious open reading frame unlikely to encode ...
3,YAL005C::chr1_1,-0.280,YAL005C,SSA1,ATPase involved in protein folding and nuclear...
4,YAL007C::chr1_1,0.290,YAL007C,ERP2,Protein that forms a heterotrimeric complex wi...


In [16]:
original_data['orf'] = original_data['ORF'].apply(lambda x: x.split(':')[0])

In [17]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [18]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [19]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Fitness defect score, SGD_orf_name, Gene, SGD_description, orf]
Index: []


In [20]:
original_data['data'] = -original_data['Fitness defect score']

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(5342, 1)

In [25]:
# Split hom and het data
nonessentials = original_data.index.values[~is_essential(original_data.index.values)]
essentials = original_data.index.values[is_essential(original_data.index.values)]

Loading SGD features from 2017-04-03.
Loading SGD features from 2017-04-03.


In [26]:
original_data_list = [original_data.loc[nonessentials,:].copy(), original_data.loc[essentials,:].copy()]

In [27]:
original_data = pd.concat(original_data_list, axis=1)

In [29]:
original_data.index.name = 'orf'

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [5342, 5341]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,5342,5341
data_type,value,value
orf,,
YAL004W,0.139,NaN
YAL005C,0.280,NaN
YAL007C,-0.290,NaN
YAL008W,-0.148,NaN
YAL009W,-2.890,NaN


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5342,5341
,data_type,value,value
gene_id,orf,,
1863,YAL004W,0.139,NaN
4,YAL005C,0.280,NaN
5,YAL007C,-0.290,NaN
6,YAL008W,-0.148,NaN
7,YAL009W,-2.890,NaN


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,5342,5341,5342,5341
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1863,YAL004W,0.139,NaN,0.205262,NaN
4,YAL005C,0.280,NaN,0.370961,NaN
5,YAL007C,-0.290,NaN,-0.298884,NaN
6,YAL008W,-0.148,NaN,-0.132010,NaN
7,YAL009W,-2.890,NaN,-3.354315,NaN


# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 22727066...
Inserting the new data...


100%|██████████| 2/2 [00:14<00:00,  7.32s/it]

Updating the data_modified_on field...
